## Installing Libraries and Mounting Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount = True)

Mounted at /content/drive


In [2]:
%%capture
!pip install https://github.com/nwhitehead/pyfluidsynth/archive/master.zip mido
!sudo apt-get install fluidsynth
!pip install pretty_midi
!pip install music21
!pip install imageio
!pip install Pillow
!pip install py_midicsv

## Importing Dependencies

In [1]:
import os
import sys
import torch
#import fluidsynth

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import pandas as pd
import numpy as np

from tqdm import tqdm
from mido import MidiFile, MidiTrack, Message

from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

import collections
import datetime
import glob
import pathlib
import pretty_midi
import seaborn as sns
import tensorflow as tf

from IPython import display
from typing import Dict, List, Optional, Sequence, Tuple

from pathlib import Path

import music21
from music21 import converter, instrument, note, chord, midi

import time

from PIL import Image

import py_midicsv

In [2]:
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten,BatchNormalization
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot
from IPython.display import clear_output
from keras.models import load_model
from numpy.random import randn
from matplotlib import pyplot

In [3]:
#sys.path.append('/content/drive/MyDrive/Music_Generation')

from midi2img import midi2image

from img2midi import image2midi

## Getting the dataset

In [11]:
path = 'C:/Users/Ody/Desktop/Work/ComputerSciencePostgraduate/ComputerSciencePostgraduate/Deep Learning/Music_Generation_Project/png_folder'
#subfolders = [f.path for f in os.scandir(path) if f.is_dir()]
os.chdir(path)
midiz = os.listdir()
midis = []
for midi in midiz:
    midis.append('' + midi)
print(len(midis))

6886


In [7]:
# sample_file = midis[1]
# pm = pretty_midi.PrettyMIDI(sample_file)
# print(sample_file)

MIDI-Unprocessed_XP_08_R1_2004_01-02_ORIG_MID--AUDIO_08_R1_2004_01_Track01_wav.midi


In [4]:
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=30):
  _SAMPLING_RATE = 16000
  waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
  # Take a sample of the generated waveform to mitigate kernel resets
  waveform_short = waveform[:seconds*_SAMPLING_RATE]
  return display.Audio(waveform_short, rate=_SAMPLING_RATE), waveform

In [9]:
#w = display_audio(pm)

## Trying to Convert Midis to Images

- Did it locally

## Get images and transform them to train dataset

In [5]:
import os
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np
path = 'C:/Users/Ody/Desktop/Work/ComputerSciencePostgraduate/ComputerSciencePostgraduate/Deep Learning/Music_Generation_Project/png_folder'
os.getcwd()
img_list = os.listdir(path)
def access_images(img_list,path,length):
    pixels = []
    imgs = []
    for i in range(length):
        if 'png' in img_list[i]:
            try:
                img = Image.open(path+'/'+img_list[i],'r')
                img = img.convert('1')
                pix = np.array(img.getdata())
                pix = pix.astype('float32')
                #pix /= 255.0
                pixels.append(pix.reshape(106,106,1))
                imgs.append(img)
            except:
                pass
    return np.array(pixels),imgs
def show_image(pix_list):
    array = np.array(pix_list.reshape(106,106), dtype=np.uint8)
    new_image = Image.fromarray(array)
    new_image.show()

pixels,imgs = access_images(img_list,path,6886)

In [6]:
np.unique(pixels)

array([  0., 255.], dtype=float32)

In [17]:
print(len(imgs))

6886


## Creating GAN model

### Defining Discriminator

In [7]:
def define_discriminator(in_shape = (106,106,1)):
    model = Sequential()
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.5))
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

### Defining Generator

In [8]:
def define_generator(latent_dim):
    model = Sequential()
    n_nodes = 128 * 53 * 53
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((53, 53, 128)))
    model.add(Dense(1024))
    model.add(Conv2DTranspose(1024, (4,4), strides=(2,2), padding='same'))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1024))
    model.add(Conv2D(1, (7,7) , padding='same',activation = 'sigmoid'))
    return model

### Synthesis

In [9]:
def define_gan(g_model, d_model):
    d_model.trainable = False
    model = Sequential()
    model.add(g_model)
    model.add(d_model)
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

## Generating Samples

In [10]:
def generate_real_samples(dataset, n_samples):
    ix = randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = ones((n_samples, 1))
    return X, y

def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input
def generate_fake_samples(g_model, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = g_model.predict(x_input)
    y = zeros((n_samples, 1))
    return X, y


## Training Routine for GAN model

In [11]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=1, n_batch=6):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    pbar = tqdm(range(n_epochs*bat_per_epo))
    for i in range(n_epochs):
        for j in range(bat_per_epo):
            X_real, y_real = generate_real_samples(dataset, half_batch)
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
            d_loss, _ = d_model.train_on_batch(X, y)
            X_gan = generate_latent_points(latent_dim, n_batch)
            y_gan = ones((n_batch, 1))
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            pbar.update(1)
            #print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))
        # if (i+1) % 10 == 0:
        #     summarize_performance(i, g_model, d_model, dataset, latent_dim)
        #     clear_output()

In [17]:
latent_dim = 100
d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)
#print(pixels.shape)
train(g_model, d_model, gan_model, np.array(pixels), latent_dim)

KeyboardInterrupt: 

## Visualizing Results

In [18]:
os.chdir('C:/Users/Ody/Desktop/Work/ComputerSciencePostgraduate/ComputerSciencePostgraduate/Deep Learning/Music_Generation_Project')

def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input
model = g_model
for i in range(1):
  latent_points = generate_latent_points(latent_dim,1)
  X = g_model.predict(latent_points)
  array = np.array(X.reshape(106,106),dtype = np.uint8)
  #array*= 255

#array.unique()

new_image = Image.fromarray(array,'L')
new_image = new_image.save('troll.png')

InvalidArgumentError: Graph execution error:

Detected at node 'sequential_1/dense_1/MatMul' defined at (most recent call last):
    File "c:\Users\Ody\anaconda3\envs\music_env\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Ody\anaconda3\envs\music_env\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\Ody\anaconda3\envs\music_env\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\Ody\anaconda3\envs\music_env\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "c:\Users\Ody\anaconda3\envs\music_env\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\Ody\anaconda3\envs\music_env\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\Ody\anacond

In [52]:
unique_elements = np.unique(array)
print(unique_elements)

[0]


## Converting to midi

In [42]:
image2midi('6000_png_10_epoch_gan_composition.png')